# # DeepFix : Deep learning for automatically fixing single-line syntax errors in C programs (using Encoder-Decoder-Attention model approach.) 

In [2]:
#from keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, LSTM, Dense , Bidirectional ,  Concatenate
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from operator import itemgetter
import pandas as pd
import pickle 
import tensorflow as tf
import numpy as np
import csv
from BahdanauAttention import AttentionLayer
from customfunction import text2sequence , sequence2tokenization , encoder_decoder_data


# Load the data & Build vocabulary on traning data ---

Dataset Description :

Each row of the dataset is buggy C program line and 'target' C line which represents fixed C program line.

In [3]:
data=pd.read_csv('G:/IISc/2nd-semester/ASE with ML/Assignment/DeepFixLite/train.csv')

input_texts = data['sourceLineTokens']
target_texts = data['targetLineTokens']

input_lines , target_lines , token2count = text2sequence(input_texts , target_texts)

(1)Setting parameters such as sentence length, vocabulory size = num_unique_tokens.

(2) Select top 1000 most frequent words and construct vocabulary only for that many words. 

(3) Download Dictionary.

In [7]:
k=1000 # select top 1000 words(most frequent)
sent_length=70
num_unique_tokens = k + 4
num_samples = len(input_lines)

token_with_count = dict(sorted(token2count.items(), key = itemgetter(1), reverse = True)[:k])
token2index = {"PAD":0, "SOS":1, "EOS":2, "OOV":3}
index2token = {0: "PAD", 1: "SOS", 2: "EOS", 3: "OOV"}
num_count = 4
for i in token_with_count:
  token2index[i] = num_count
  index2token[num_count] = i
  num_count += 1

# pickle_out1 = open("token2index.pickle" , "wb")
# pickle_out2 = open("index2token.pickle" , "wb")

# pickle.dump(token2index , pickle_out1)
# pickle.dump(index2token , pickle_out2)

# pickle_out1.close()
# pickle_out2.close()

In [8]:
max_encoder_seq_length = max([len(eval(item)) for item in input_lines]) #154
max_decoder_seq_length = max([len(eval(item)) for item in target_lines]) #169

print("size of the original vocabulary =",len(token2count))
print("size of the updated vocabulary =",len(token_with_count))
print("max_encoder_seq_length =",max_encoder_seq_length)
print("max_decoder_seq_length =",max_decoder_seq_length)
print("number of samples =",num_samples )


size of the original vocabulary = 5205
size of the updated vocabulary = 1000
max_encoder_seq_length = 154
max_decoder_seq_length = 169
number of samples = 14643


# Generating fixed size sequences which can be given input to neural networks

(1) Tokenise input sequence 

(2) Add padding to make equal size sequence  

Above two functionalities implemented in sequence2tokenization function.

In [9]:
padded_input_texts = sequence2tokenization(input_lines , token2index , sent_length)
padded_target_texts = sequence2tokenization(target_lines , token2index , sent_length)

print(padded_input_texts[0])

[ 1 21  4 82 48  8 42 11 17  5  2  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


 # Encoder Input data, decoder input data are inputs that need to be given to LSTM's networks for encoding and decoding.

In [10]:
encoder_input_data , decoder_input_data , decoder_target_data = encoder_decoder_data(padded_input_texts , padded_target_texts ,num_samples ,sent_length ,num_unique_tokens )


# Pre_process the validation data(same step as we did previously on training data) for parameter tunning.

In [11]:
valid_data=pd.read_csv('G:/IISc/2nd-semester/ASE with ML/Assignment/DeepFixLite/valid_complete.csv')

input_valid_texts = valid_data['sourceLineTokens']
target_valid_texts = valid_data['targetLineTokens']
num_valid_samples = len(input_valid_texts)
padded_input_valid_texts = sequence2tokenization(input_valid_texts , token2index , sent_length)
padded_target_valid_texts = sequence2tokenization(target_valid_texts , token2index , sent_length)

encoder_input_valid_data ,decoder_input_valid_data ,decoder_target_valid_data = encoder_decoder_data(padded_input_valid_texts , padded_target_valid_texts ,num_valid_samples ,sent_length ,num_unique_tokens )



# Design Encoder-decoder-Attention model.

In [14]:
latent_dim=256
batch_size=64
epochs = 10
emb_size = 50

# Encoder 

encoder_inputs = Input(shape=(sent_length,)) 
enc_emb = Embedding(num_unique_tokens, 1024)(encoder_inputs)

# Bidirectional lstm layer
enc_lstm1 = Bidirectional(LSTM(256,return_sequences=True,return_state=True))
encoder_outputs1, forw_state_h, forw_state_c, back_state_h, back_state_c = enc_lstm1(enc_emb)

final_enc_h = Concatenate()([forw_state_h,back_state_h])
final_enc_c = Concatenate()([forw_state_c,back_state_c])

encoder_states =[final_enc_h, final_enc_c]

# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(num_unique_tokens, 1024) 
dec_emb = dec_emb_layer(decoder_inputs)
#LSTM using encoder_states as initial state
decoder_lstm = LSTM(512, return_sequences=True, return_state=True) 
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

#Attention Layer
attention_layer = AttentionLayer()
attention_result, attention_weights = attention_layer([encoder_outputs1, decoder_outputs])

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attention_result])

#Dense layer
decoder_dense = Dense(num_unique_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)


# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 70)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 70, 1024)     1028096     input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 70, 512), (N 2623488     embedding_2[0][0]                
______________________________________________________________________________________________

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [20]:
history1= model.fit(
    [encoder_input_data, decoder_input_data],decoder_target_data,
    batch_size=batch_size,
    epochs=3,
    #validation_split=0.2,
    validation_data=([encoder_input_valid_data,decoder_input_valid_data],decoder_target_valid_data)
)

Epoch 1/3
229/229 [==============================] - 2519s 11s/step - loss: 0.4684 - accuracy: 0.9020 - val_loss: 0.3322 - val_accuracy: 0.9258
Epoch 2/3
229/229 [==============================] - 2373s 10s/step - loss: 0.2876 - accuracy: 0.9365 - val_loss: 0.2545 - val_accuracy: 0.9455
Epoch 3/3
229/229 [==============================] - 2537s 11s/step - loss: 0.2096 - accuracy: 0.9521 - val_loss: 0.1888 - val_accuracy: 0.9580


In [21]:
tf.keras.models.save_model(model, "G:/IISc/2nd-semester/ASE with ML/Assignment/DeepFixLite/Attention_model")

INFO:tensorflow:Assets written to: G:/IISc/2nd-semester/ASE with ML/Assignment/DeepFixLite/Attention_model\assets


INFO:tensorflow:Assets written to: G:/IISc/2nd-semester/ASE with ML/Assignment/DeepFixLite/Attention_model\assets
